### Get the Personalize boto3 Client

In [ ]:
import boto3

import json
import numpy as np
import pandas as pd
import time

!wget -N https://s3-us-west-2.amazonaws.com/personalize-cli-json-models/personalize.json
!wget -N https://s3-us-west-2.amazonaws.com/personalize-cli-json-models/personalize-runtime.json
!aws configure add-model --service-model file://`pwd`/personalize.json --service-name personalize
!aws configure add-model --service-model file://`pwd`/personalize-runtime.json --service-name personalize-runtime

personalize = boto3.client(service_name='personalize', endpoint_url='https://personalize.us-west-2.amazonaws.com', region_name='us-west-2')
personalize_runtime = boto3.client(service_name='personalize-runtime', endpoint_url='https://personalize-runtime.us-west-2.amazonaws.com', region_name='us-west-2')

### Specify a Bucket and Data Output Location

In [ ]:
bucket = "personalize-peerjako"            # replace with the name of your S3 bucket
filename = "DEMO-movie-lens-100k.csv"  # replace with a name that you want to save the dataset under

### Download, Prepare, and Upload Training Data

#### Download and Explore the Dataset

Once we’ve downloaded and unzipped the dataset, let’s load the ‘u.data’ file

In [ ]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'])
pd.set_option('display.max_rows', 5)
data

#### Prepare and Upload Data

Lets apply the following processing:

* Keep only movies rated 4 and above, and drop the ratings columns: we just want our model to recommend movies that users should really like.
* Rename columns to the names used in the schema.

All of this is easily achieved with the Pandas Python library, the Swiss Army knife for columnar data processing. While we’re at it, we’ll also upload the processed file to an Amazon S3 bucket.


In [ ]:
data = data[data['RATING'] > 3.6]                # keep only movies rated 3.6 and above
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
data.to_csv(filename, index=False)

boto3.Session().resource('s3').Bucket(bucket).Object(filename).upload_file(filename)

### Create Schema
The first step is to create an Avro schema for this dataset. This is pretty straightforward, we just need to use some of the keywords defined in Amazon Personalize.

In [ ]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "DEMO-schema",
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

### Create and Wait for Dataset Group

#### Create Dataset Group

First, we need to create a dataset group containing the user-item dataset as well as its schema. Let’s do this with the AWS Python SDK.

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "DEMO-dataset-group"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

#### Wait for Dataset Group to Have ACTIVE Status

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

### Create Dataset

In this simple example, we’ll import data on-demand. It’s also possible to schedule import jobs in order to load new data regularly. We need to pass a role allowing data to be read from the Amazon S3 bucket.

In [ ]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "DEMO-dataset",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

### Prepare, Create, and Wait for Dataset Import Job

#### Attach policy to S3 bucket

In [ ]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy));

#### Create S3 Read Only Access Role

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeS3Role"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
);

iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
);

iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = "arn:aws:iam::aws:policy/CloudWatchFullAccess"
);


role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

#### Create Dataset Import Job

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "DEMO-dataset-import-job",
    datasetArn = dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket, filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

#### Wait for Dataset Import Job to Have ACTIVE Status

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

### Select Recipe
Once datasets have been imported, we need to select a recipe to cook our recommendation model. A recipe is much more than an algorithm: it also includes predefined feature transformation, initial parameters for the algorithm as well as automatic model tuning. Thus, recipes remove the need to have expertise in personalization.

Amazon Personalize comes with several recipes suitable for different use cases, and advanced users can also add their own recipes.

Here’s the list of available recipes.

In [52]:
list_recipes_response = personalize.list_recipes()
recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn" # aws-hrnn selected for demo purposes
list_recipes_response

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
   'content-length': '989',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Tue, 07 May 2019 08:02:09 GMT',
   'x-amzn-requestid': 'ab743391-595b-4885-ab70-34ee61f54197'},
  'HTTPStatusCode': 200,
  'RequestId': 'ab743391-595b-4885-ab70-34ee61f54197',
  'RetryAttempts': 0},
 u'recipes': [{u'creationDateTime': datetime.datetime(2018, 11, 26, 0, 0, tzinfo=tzlocal()),
   u'lastUpdatedDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal()),
   u'name': u'aws-hrnn',
   u'recipeArn': u'arn:aws:personalize:::recipe/aws-hrnn',
   u'status': u'ACTIVE'},
  {u'creationDateTime': datetime.datetime(2018, 11, 26, 0, 0, tzinfo=tzlocal()),
   u'lastUpdatedDateTime': datetime.datetime(1970, 1, 1, 0, 0, tzinfo=tzlocal()),
   u'name': u'aws-hrnn-coldstart',
   u'recipeArn': u'arn:aws:personalize:::recipe/aws-hrnn-coldstart',
   u'status': u'ACTIVE'},
  {u'creationDateTime': datetime.datetime(2018, 11, 26, 0, 0, tzinfo

Recommendation experts will certainly enjoy the flexibility that they bring, but what about developers who are new to the topic?

As mentioned earlier, Amazon Personalize supports AutoML, a new technique that automatically searches for the most optimal recipe.

### Create and Wait for Solution

#### Create Solution

In [ ]:
create_solution_response = personalize.create_solution(
    name = "DEMO-solution",
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

#### Create Solution Version

In [ ]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

#### Wait for Solution Version to Have ACTIVE Status

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

#### Get Metrics of Solution

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

### Create and Wait for Campaign

If we’re happy with the model, we can now create a campaign in order to deploy it. It will be updated automatically every time the solution is deployed.

#### Create Campaign

In [ ]:
create_campaign_response = personalize.create_campaign(
    name = "DEMO-campaign",
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 1
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

#### Wait for Campaign to Have ACTIVE Status

In [ ]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

### Get Recommendations

#### Select a User and an Item

In [50]:
items = pd.read_csv('./ml-100k/u.item', sep='|', usecols=[0,1], header=None)
items.columns = ['ITEM_ID', 'TITLE']

user_id, item_id, _ = data.sample().values[0]
item_title = items.loc[items['ITEM_ID'] == item_id].values[0][-1]
print("USER: {}".format(user_id))
print("ITEM: {}".format(item_title))

items

USER: 826
ITEM: Batman (1989)


,ITEM_ID,TITLE
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
...,...,...
1680,1681,You So Crazy (1994)
1681,1682,Scream of Stone (Schrei aus Stein) (1991)


#### Call GetRecommendations

In [51]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id),
    itemId = str(item_id)
)

item_list = get_recommendations_response['itemList']
title_list = [items.loc[items['ITEM_ID'] == np.int(item['itemId'])].values[0][-1] for item in item_list]

print("Recommendations: {}".format(json.dumps(title_list, indent=2)))

Recommendations: [
  "Ace Ventura: Pet Detective (1994)", 
  "It Could Happen to You (1994)", 
  "Ghost (1990)", 
  "Pretty Woman (1990)", 
  "Tin Men (1987)", 
  "Dirty Dancing (1987)", 
  "Nightmare on Elm Street, A (1984)", 
  "Demolition Man (1993)", 
  "Grease (1978)", 
  "Nightmare Before Christmas, The (1993)", 
  "Tales From the Crypt Presents: Demon Knight (1995)", 
  "Home Alone (1990)", 
  "Jaws 2 (1978)", 
  "Tommy Boy (1995)", 
  "Star Trek: The Motion Picture (1979)", 
  "Virtuosity (1995)", 
  "French Kiss (1995)", 
  "Nell (1994)", 
  "Carrie (1976)", 
  "Batman (1989)", 
  "Speechless (1994)", 
  "Waterworld (1995)", 
  "Doors, The (1991)", 
  "Jumanji (1995)", 
  "Basic Instinct (1992)"
]
